In [27]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
%cd 'gdrive/MyDrive/Colab Notebooks/CHL/semester-project'
%ls

[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab Notebooks/CHL/semester-project'
/content/gdrive/MyDrive/Colab Notebooks/CHL/semester-project
ANN.ipynb  dataset/


## Libraries


In [29]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


In [30]:
np.random.seed(42)

## Data Processing

In [31]:
df = pd.read_csv("dataset/mutationsComplete.csv", index_col=False)
df = df.drop("Unnamed: 0", axis=1)
# print(df.head())

# exit()

df = df.loc[:, ~df.columns.isin(['AccessionNumber', '1stpfs event', 'dpfs', 'dos'])]
# df = df[df["safety"]==1] #select safety analysis
# df = df.drop('PatientCode',axis=1) #drop one row with Nan value
# df = df[~df.isin([-99]).any(axis=1)] #drop any rows with -99 value
# # For now, remove all rows with Nan values
# # remove rows with NaN values, it's actually only one row
# df = df.dropna()

print(len(df))

296


In [32]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
# train, validation = train_test_split(train, test_size=0.1, random_state=42)

In [33]:
# Separate the target variable from the features
y_train = train['os event']
X_train = train.drop('os event', axis=1)
# y_val = validation["os event"]
# X_val = validation.drop("os event", axis=1)
y_test = test['os event']
X_test = test.drop('os event', axis=1)

## Normalizing Data

In [34]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## ANN Model

In [35]:
early_stopping_monitor = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience=5,
    restore_best_weights=True
)

In [36]:
num_features = X_train.shape[1]

def create_model():
  model = tf.keras.Sequential()

  model.add(tf.keras.Input((num_features,)))
  model.add(tf.keras.layers.Dense(8))
  model.add(tf.keras.layers.Activation("relu"))
  model.add(tf.keras.layers.Dense(16))
  model.add(tf.keras.layers.Activation("relu"))
  model.add(tf.keras.layers.Dense(16))
  model.add(tf.keras.layers.Activation("relu"))
  model.add(tf.keras.layers.Dense(8))
  model.add(tf.keras.layers.Activation("relu"))
  model.add(tf.keras.layers.Dense(1))
  model.add(tf.keras.layers.Activation("sigmoid"))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [37]:
model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
batch_size = [4, 8, 16, 32, 64]
epochs = [5, 7, 10, 12, 15, 17, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)


# model_history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

<ipython-input-37-7a9a9f458f0e>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=1)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/5
8/8 [==============================] - 2s 5ms/step - loss: 0.6927 - accuracy: 0.6144
Epoch 2/5
8/8 [==============================] - 0s 4ms/step - loss: 0.6908 - accuracy: 0.6737
Epoch 3/5
8/8 [==============================] - 0s 4ms/step - loss: 0.6870 - accuracy: 0.6737
Epoch 4/5
8/8 [==============================] - 0s 4ms/step - loss: 0.6791 - accuracy: 0.6737
Epoch 5/5
8/8 [==============================] - 0s 4ms/step - loss: 0.6697 - accuracy: 0.6737
Best: 0.681976 using {'batch_size': 32, 'epochs': 5}
0.673429 (0.054205) with: {'batch_size': 4, 'epochs': 5}
0.656497 (0.047484) with: {'batch_size': 4, 'epochs': 7}
0.622471 (0.069245) with: {'batch_size': 4, 'epochs': 10}
0.614519 (0.019442) with: {'batch_size': 4, 'epochs': 12}
0.601753 (0.032470) with: {'batch_size': 4, 'epochs': 15}
0.631234 (0.022756) with: {'batch_size': 4, 'epochs': 17}
0.635562 (0.007333) with: {'batch_size': 4, 'epochs': 20}
0.673429 (0.054205) with: {'batch_size': 8, 'epochs': 5}
0.631234 (0

In [38]:
# model.summary()
print("Tuned ANN Parameters: {}".format(grid_result.best_params_))
print("Best score on validation is {}".format(grid_result.best_score_))
grid_predictions = grid_result.predict(X_test)

Tuned ANN Parameters: {'batch_size': 32, 'epochs': 5}
Best score on validation is 0.6819755434989929
2/2 [==============================] - 0s 3ms/step


In [39]:
accuracy_score(y_test, grid_predictions)

0.7166666666666667